Let's make the necessary imports for our implementation

In [111]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


In [112]:
house_data = pd.read_csv("data.csv")
house_data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,2014-05-02 00:00:00,313000.0,3.0,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,2014-05-02 00:00:00,2384000.0,5.0,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA
2,2014-05-02 00:00:00,342000.0,3.0,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,2014-05-02 00:00:00,420000.0,3.0,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,2014-05-02 00:00:00,550000.0,4.0,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA


First, we are going to implement simple linear regression.
We will use the prices as our response and bedrooms to be our features

So, following the formula for simple linear regression, our model is going to represent:
 $$price = \beta_0 + \beta_1 \cdot \, bedrooms \, + \epsilon_i $$

 where $$ \epsilon $$ is our irreducible error term but can be ommitted since we don't know the true values for this


It is common practice to represent the regression model via matrix multiplication as it is easier to work with so now need to make a design matrix in the form of
$$ y = X \beta  $$
where $$ X $$ represents a design matrix containing rows of ones for the first column which accounts for intercept/bias term and all subsquent columns are our features
and $$\beta$$ is a vector containing our model's weights, one for each variable

In [113]:
X = pd.DataFrame({'intercept' : np.ones(house_data.shape[0]), 'bedrooms' : house_data['bedrooms'], 
                  'bathrooms' : house_data['bathrooms']})
X = X.to_numpy()

Now that we have a design matrix our next goal is to get estimates for our weights via gradient descent!

In order to use gradient descent we have to define our cost function. We will use MSE, mean squared error, which is defined as:
$$
\text{MSE}(\beta) = \frac{1}{n} \| X\beta - y \|^2
$$
This is the matrix form of MSE and not the standard formula you would see


We must take the gradient with respect to $$\beta$$
which ends up looking like:
$$\nabla J(\boldsymbol{\beta}) = \frac{1}{n} \mathbf{X}^\top (\mathbf{X} \boldsymbol{\beta} - \mathbf{y})$$




Let's define a function for gradient descent

In [114]:
def gradient_descent(y, X, beta, n, alpha, iterations):
    for i in range(0, 100):
        gradient =  X.T @ (X @ beta - y)/ n
        beta = beta - alpha * gradient
        
    return beta

Let's define the necessary variables to use the gradient_descent function, our response is going to be our prices and we'll initalize our weights to random values. We are also going to split the data into training and test sets

In [115]:
y = house_data['price']
y = y.to_numpy()
beta = np.ones(X.shape[1])
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
n = X_train.shape[0]

In [116]:
beta = gradient_descent(y_train, X_train, beta, n, 0.1, 100)

In [117]:
print(beta)

[ -1639.4363617   22469.94994039 220044.25999952]


It's hard to tell how many iterations we should use for gradient descent, so to help gauge how many iterations we need, we can take a look at our cost function every couple of iterations of our choosing. 
If we see that the cost function is not decreasing by a certain threshold of our liking, then we can put a stop to the iterations early, reducing the amount of 
compute used.
So let's define the cost function in this case, MSE.
Afterwards we will update our gradient descent function to include a stopping condition using the MSE




In [118]:
def mean_squared_error(y, X, beta, n):
    mse = ((X @ beta - y).T @ (X @ beta - y)) / n
    return mse

In [119]:
def gradient_descent(y, X, beta, n, alpha, iterations, tolerance=1e-5, check_every=100):
    prev_mse = mean_squared_error(y, X, beta, n)
    
    for i in range(iterations):
        gradient = (1 / n) * X.T @ (X @ beta - y)
        beta = beta - alpha * gradient

        # Check MSE change every check_every steps
        if i % check_every == 0:
            current_mse = mean_squared_error(y, X, beta, n)
            improvement = abs(prev_mse - current_mse)

            print(f"Iteration {i}: MSE = {current_mse:.4f}, MSE decreased by = {improvement:.6f}")

            if improvement < tolerance:
                print(f"Early stopping at iteration {i} — MSE improvement below threshold ({tolerance})")
                return beta

            prev_mse = current_mse  # Update for next check

    print(f"Completed full {iterations} iterations. Final MSE: {mean_squared_error(y, X, beta, n):.4f}")
    return beta
beta = gradient_descent(y,X,beta, n, 0.01, 100000)


Iteration 0: MSE = 423321595661.6596, MSE decreased by = 1573807.330444
Iteration 100: MSE = 423314630111.8835, MSE decreased by = 6965549.776062
Iteration 200: MSE = 423312349750.8951, MSE decreased by = 2280360.988403
Iteration 300: MSE = 423311256831.1976, MSE decreased by = 1092919.697571
Iteration 400: MSE = 423310651966.1788, MSE decreased by = 604865.018738
Iteration 500: MSE = 423310263571.7040, MSE decreased by = 388394.474854
Iteration 600: MSE = 423309983438.6240, MSE decreased by = 280133.080017
Iteration 700: MSE = 423309766207.7997, MSE decreased by = 217230.824219
Iteration 800: MSE = 423309591078.8568, MSE decreased by = 175128.942993
Iteration 900: MSE = 423309447161.5002, MSE decreased by = 143917.356506
Iteration 1000: MSE = 423309327819.0140, MSE decreased by = 119342.486267
Iteration 1100: MSE = 423309228440.5083, MSE decreased by = 99378.505676
Iteration 1200: MSE = 423309145527.7907, MSE decreased by = 82912.717590
Iteration 1300: MSE = 423309076292.2950, MSE dec

Now we can finally do our predictions

In [121]:
print(beta)
predictions = X_test @ beta

[  4391.97937421  19231.34338782 223141.57667692]


In [122]:
print(predictions)

[619939.95122996 602617.24383637 787296.13373765 ... 658402.6380056
 471815.11211009 658402.6380056 ]


Now we can use Scikit-learn's MAE function to see how our model did 

In [127]:
mean_absolute_error(y_test, predictions)

230680.8267999549

This value indicates that, on average, our model is off from the true values by the above amount. This means our model is not particulary performing well. However, we completed our goal of implementing a basic implementation of linear regression using gradient descent.

If we wanted to further improve this model, we would include more features and make sure our implementation can handle that. We would through the process of feature selection determing what features effect the model the most and build a model with those features. It would be worth scaling our data since our features have varying scales